# Toronto neighborhood analysis for an Indian family to live in 

## 1. Introduction

<b> City of Toronto </b> is the most populous city in Canada. It is home to various ethnic groups such as Canadian, English, Chinese, Indian, Italian etc. It is recognized as one of the most multicultural and cosmopolitan cities in the world.

Toronto is an international center of business, finance and culture. The business and employment opportunities in Toronto leads to temporary or permanent immigration of many families. There is also a lot of internal migration from within Canada to Toronto. Reasons can be economic, arts, better standard of living, etc.

Migrating to a new city is not a simple task. One has to consider multiple facets while choosing a neighborhood to settle ex: Proximity to schools, colleges, hospitals, banks, hotels, etc. These are some of the basic important factors which a person should consider whenever he/she tries to relocate to a new city.

In this report, we will focus on Indian families that are planning to move to the great city of Toronto. 

## 2. Problem Statement

<h3>Enable an Indian family to select a neigbhorhood to relocate in the city of Toronto.</h3>
    
By reading this report, an Indian family can compare different neighborhoods and they should get an idea of the neighborhoods that they can consider for relocating in Toronto.

## 3. Data

**A.** We need to identify all the neighborhoods in the city of Toronto. Neighborhood details according to postal codes are available on the following wiki page https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M .




In [20]:
# @hidden_cell
import pandas as pd # library for data analsysis

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [21]:
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M'

df = pd.read_html(url, header=0)[0] # read an html table and load it into a dataframe
print ("Neighborhoods:")
df.head()


Neighborhoods:


,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


**B.** We need to identify coordinates of each neighborhood. These are available at : http://cocl.us/Geospatial_data. 
This is a csv file that has the geographical coordinates of each postal code of Toronto



In [22]:
geo_data = pd.read_csv('http://cocl.us/Geospatial_data') # read csv having coordinate for postal code
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


**C.** We need to identify important elements of each neighborhood such as schools, hospitals, parks etc. Since we are focusing on Indian family, it will be great if we could identify Hotels serving Indian food as well. This information can be accessed using third party Foursquare APIs.

**Foursquare** is a social location service that allows users to explore the world around them. Based on latitude and longitude, it will provide nearby services such as hotels, banks, stores, specific type of restaurants etc. A free Foursquare developer account has already been created for use.

Here, we will explore each neighborhood for radius of 1 Km based on lattitude and longitude. Following is an example of venues in each neigbhorhood.

|	|Neighbourhood|Neighbourhood Latitude|Neighbourhood Longitude|Venue|Venue Latitude|Venue Longitude|Venue Category|
|-|-------------|----------------------|-----------------------|-----|--------------|---------------|--------------|
|0|The Beaches|43.676357|-79.293031|Glen Manor Ravine|43.676821|-79.293942|Trail|
|1|The Beaches|43.676357|-79.293031|The Big Carrot Natural Food Market|43.678879|-79.297734|Health Food Store|
|2|The Beaches|43.676357|-79.293031|Grover Pub and Grub|43.679181|-79.297215|Pub|
|3|The Beaches|43.676357|-79.293031|Upper Beaches|43.680563|-79.292869|Neighborhood|
|4|The Danforth West,Riverdale|43.679557|-79.352188|Pantheon|43.677621|-79.351434|Greek Restaurant|


Using the essential venue categories, we can divide the neighborhoods into different segments: **Excellent, Good, Moderate, Poor**.
This will help an individual make a decision on which neighborhood to select.